# `VKS document with llama3`


In [1]:
%load_ext autoreload
%autoreload 2

## On its own


In [2]:
from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.converters import PDFMinerToDocument
from haystack.components.preprocessors import DocumentCleaner
from haystack.components.preprocessors import DocumentSplitter
from haystack.components.writers import DocumentWriter

In [3]:
raw_document_store = InMemoryDocumentStore()

pipeline = Pipeline()
pipeline.add_component("converter", PDFMinerToDocument())
pipeline.add_component("cleaner", DocumentCleaner())
pipeline.add_component("splitter", DocumentSplitter(split_by="sentence", split_length=5))
pipeline.add_component("writer", DocumentWriter(document_store=raw_document_store))

pipeline.connect("converter", "cleaner")
pipeline.connect("cleaner", "splitter")
pipeline.connect("splitter", "writer")

🚅 Components
  - converter: PDFMinerToDocument
  - cleaner: DocumentCleaner
  - splitter: DocumentSplitter
  - writer: DocumentWriter
🛤️ Connections
  - converter.documents -> cleaner.documents (List[Document])
  - cleaner.documents -> splitter.documents (List[Document])
  - splitter.documents -> writer.documents (List[Document])

In [4]:
file_names = ["sample.pdf"]
pipeline.run({"converter": {"sources": file_names}})

{'writer': {'documents_written': 690}}

In [5]:
from haystack.components.embedders import SentenceTransformersDocumentEmbedder

In [6]:
document_store = InMemoryDocumentStore()

Initalize a Document Embedder

In [7]:
doc_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
doc_embedder.warm_up()

In [8]:
docs_with_embeddings = doc_embedder.run(raw_document_store.filter_documents())
document_store.write_documents(docs_with_embeddings["documents"])

Batches:   0%|          | 0/22 [00:00<?, ?it/s]

690

In [9]:
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.builders import ChatPromptBuilder
from haystack.dataclasses import ChatMessage
from haystack_integrations.components.generators.ollama import OllamaChatGenerator
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

In [10]:
text_embedder = SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")

In [11]:
template = [ChatMessage.from_user("""
Given the following information, answer the question.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{question}}
Answer:
""")]

prompt_builder = ChatPromptBuilder(template=template)

In [12]:
chat_generator = OllamaChatGenerator(
    model="llama3.1:8b",
    streaming_callback=lambda chunk: print(chunk.content, end="", flush=True),
    url = "http://localhost:11434",
    generation_kwargs={
        "num_predict": 100,
        "temperature": 0.9})

Initialize retriever

In [13]:
retriever = InMemoryEmbeddingRetriever(document_store)

In [14]:
basic_rag_pipeline = Pipeline()
# Add components to your pipeline
basic_rag_pipeline.add_component("text_embedder", text_embedder)
basic_rag_pipeline.add_component("retriever", retriever)
basic_rag_pipeline.add_component("prompt_builder", prompt_builder)
basic_rag_pipeline.add_component("llm", chat_generator)

In [15]:
# Now, connect the components to each other
basic_rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
basic_rag_pipeline.connect("retriever", "prompt_builder")
basic_rag_pipeline.connect("prompt_builder.prompt", "llm.messages")

🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: ChatPromptBuilder
  - llm: OllamaChatGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.messages (List[ChatMessage])

# Asking question

In [16]:
question = "What is VKS?"

response = basic_rag_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})

print(response["llm"]["replies"][0].text)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

VKS (VNGCloud Kubernetes Service) is a managed service on VNGCloud that simplifies the deployment and management of container-based applications. It is based on the open-source platform Kubernetes, developed by Google, which is widely used for managing and deploying containerized applications in distributed environments.VKS (VNGCloud Kubernetes Service) is a managed service on VNGCloud that simplifies the deployment and management of container-based applications. It is based on the open-source platform Kubernetes, developed by Google, which is widely used for managing and deploying containerized applications in distributed environments.


In [17]:
question = "Compare VKS private clusters and VKS public clusters"

response = basic_rag_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})

print(response["llm"]["replies"][0].text)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Here's the comparison between VKS Private Clusters and VKS Public Clusters:

**Key differences:**

1. **Security**: Private Clusters provide completely private connections between nodes, control plane, clients, and other services in VNG Cloud. Public Clusters use Public IP addresses to communicate.
2. **Access Control**: Private Clusters have stricter access control, ensuring compliance with security regulations and data privacy.
3. **IP Addresses**: Private Clusters use Private IP addresses, while Public ClHere's the comparison between VKS Private Clusters and VKS Public Clusters:

**Key differences:**

1. **Security**: Private Clusters provide completely private connections between nodes, control plane, clients, and other services in VNG Cloud. Public Clusters use Public IP addresses to communicate.
2. **Access Control**: Private Clusters have stricter access control, ensuring compliance with security regulations and data privacy.
3. **IP Addresses**: Private Clusters use Private IP 

In [18]:
question = "What is vngcloud-blockstorage-csi-driver?"

response = basic_rag_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})

print(response["llm"]["replies"][0].text)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

VNGCloud BlockStorage CSI (Container Storage Interface) Driver Plugin. 

This is a Kubernetes plugin that provides persistent storage to applications, allowing them to access block-level storage resources from within the cluster. The VNGCloud BlockStorage CSI Driver Plugin has been upgraded in recent updates to fix errors and improve system reliability.VNGCloud BlockStorage CSI (Container Storage Interface) Driver Plugin. 

This is a Kubernetes plugin that provides persistent storage to applications, allowing them to access block-level storage resources from within the cluster. The VNGCloud BlockStorage CSI Driver Plugin has been upgraded in recent updates to fix errors and improve system reliability.


In [19]:
question = "I want to create a pvc of 20Gi using vngcloud-blockstorage-csi-driver"

response = basic_rag_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})

print(response["llm"]["replies"][0].text)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

You can create a Persistent Volume Claim (PVC) of 20Gi using `vngcloud-blockstorage-csi-driver` by running the following command:

```yaml
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: my-pvc
spec:
  accessModes:
    - ReadWriteOnce
  resources:
    requests:
      storage: 20Gi
  storageClassName: vngcloud-blockstorage-csi-driver
```

You can also specifyYou can create a Persistent Volume Claim (PVC) of 20Gi using `vngcloud-blockstorage-csi-driver` by running the following command:

```yaml
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: my-pvc
spec:
  accessModes:
    - ReadWriteOnce
  resources:
    requests:
      storage: 20Gi
  storageClassName: vngcloud-blockstorage-csi-driver
```

You can also specify


In [20]:
question = "How to use volume snapshot in VKS cluster"

response = basic_rag_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})

print(response["llm"]["replies"][0].text)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

According to the provided information, to use volume snapshot in a VKS (VNG Cloud Storage) cluster:

1. Create a VolumeSnapshotClass:
```yaml
apiVersion: v1
kind: VolumeSnapshotClass
metadata:
  name: my-snapshot-storage-class
driver: bs.csi.vngcloud.vn
deletionPolicy: Delete
parameters:
  force-create: "false"
```
2. Create a VolumeSnapshot for the desired PersistentVolumeClaim (PVC):
```According to the provided information, to use volume snapshot in a VKS (VNG Cloud Storage) cluster:

1. Create a VolumeSnapshotClass:
```yaml
apiVersion: v1
kind: VolumeSnapshotClass
metadata:
  name: my-snapshot-storage-class
driver: bs.csi.vngcloud.vn
deletionPolicy: Delete
parameters:
  force-create: "false"
```
2. Create a VolumeSnapshot for the desired PersistentVolumeClaim (PVC):
```
